# **YoloV3(TensorFlow) 모델 실습**

## **1. 사전 환경 세팅**

### **1-1. TensorFlow 정상 설치 확인**

In [ ]:
!conda create -n day14-1 python=3.9
!conda activate day14-1

In [ ]:
!pip install -r requirements.txt
!pip install jupyter notebook
!python -m ipykernel install --user --name day14-1 --display-name day14-1

In [ ]:
import tensorflow as tf
tf.__version__

### **1-2. Darknet weight 파일로 변환 처리**

In [ ]:
!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.tf
!python convert.py --weights ./data/yolov3-tiny.weights --output ./checkpoints/yolov3-tiny.tf --tiny

## **2. Detector 정의**

In [ ]:
import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

In [ ]:
# absl 라이브러리를 사용한 커맨드라인 플래그(옵션) 정의
flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf',
                    'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

# 앱 초기화 및 플래그 파싱
app._run_init(['yolov3'], app.parse_flags_with_usage)

In [ ]:
# GPU 설정을 통해 사용 가능한 GPU 장치를 확인하고, GPU 메모리 활성화
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## **3. pre-trained weight를 활용한 detection 테스트**

In [ ]:
# 테스트할 이미지 입력 경로 설정
FLAGS.image = 'data/meme.jpg'

# YOLOv3 기본 또는 tiny 모델 파이프라인 선택
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)
      
# 모델 weight 파일 로드
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

# 클래스 이름 로드
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

# 입력 이미지 로드
img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3)

# 이미지 전처리
img = tf.expand_dims(img_raw, 0) # 이미지 차원 확장 (배치 차원 추가)
img = transform_images(img, FLAGS.size) # 이미지를 모델 입력 크기에 맞게 리사이즈 및 정규화

In [ ]:
# 모델 예측 수행
t1 = time.time() # 시작 시간 기록
boxes, scores, classes, nums = yolo(img) # 이미지에서 객체 탐지 수행
t2 = time.time() # 종료 시간 기록
logging.info('time: {}'.format(t2 - t1))

# 탐지된 객체 정보 출력
logging.info('detections:')
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                        np.array(scores[0][i]),
                                        np.array(boxes[0][i])))

In [ ]:
# 탐지 결과 이미지로 표시
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

# Jupyter Notebook에서 이미지 표시
from IPython.display import Image, display
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))

## **4. 신규 학습 진행**

### **4-1. 데이터 전처리**

In [ ]:
# raw 데이터(tar) 압축 해제
# !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2009/VOCtrainval_11-May-2009.tar -O ./data/voc2009_raw.tar
!mkdir -p ./data/voc2009_raw
!tar -xf ./data/voc2009_raw.tar -C ./data/voc2009_raw

In [ ]:
# YOLOv3 학습을 위한 voc2012 학습 데이터 전처리 수행
!python3 tools/voc2012.py \
  --data_dir './data/voc2009_raw/VOCdevkit/VOC2009' \
  --split train \
  --output_file ./data/voc_train.tfrecord

# YOLOv3 학습을 위한 voc2012 검증 데이터 전처리 수행
!python3 tools/voc2012.py \
  --data_dir './data/voc2009_raw/VOCdevkit/VOC2009' \
  --split val \
  --output_file ./data/voc_val.tfrecord

In [ ]:
!conda install -y -c nvidia cuda-nvcc

In [ ]:
# 학습 파라미터 설정 및 학습 시작
!python3 train.py \
	--dataset ./data/voc_train.tfrecord \
	--val_dataset ./data/voc_val.tfrecord \
	--classes ./data/voc2012.names \
	--num_classes 20 \
	--mode fit --transfer darknet \
	--batch_size 8 \
	--epochs 3 \
	--weights ./checkpoints/yolov3.tf \
	--weights_num_classes 80 

## **5. 신규 학습 weight를 활용한 추론**

In [ ]:
FLAGS.num_classes = 20 # 모델이 인식할 클래스 수를 정의
FLAGS.classes = 'data/voc2012.names'  # 클래스 이름이 정의된 파일 경로
FLAGS.weights = 'checkpoints/yolov3_train_3.tf'  # 학습된 가중치 파일 경로
FLAGS.image = 'data/meme.jpg'  # 추론에 사용할 입력 이미지 파일 경로

In [ ]:
# 3 epoch로 충분한 학습이 이루어지지 않아 낮은 threshold로 설정 및 추론 진행
FLAGS.yolo_iou_threshold = 0.1  # 예측된 바운딩 박스 간의 IoU 임계값 설정
FLAGS.yolo_score_threshold = 0.1  # 예측된 바운딩 박스의 점수 임계값 설정

In [ ]:
# 모델을 YOLOv3 Tiny 또는 기본 YOLOv3로 설정
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)  # YOLOv3 Tiny 모델 초기화
else:
    yolo = YoloV3(classes=FLAGS.num_classes)  # YOLOv3 기본 모델 초기화

# 학습된 가중치 로드
yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')  # 가중치가 로드되었음을 로그에 기록

# 클래스 이름을 로드
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')  # 클래스 이름이 로드되었음을 로그에 기록

# 입력 이미지를 읽고 디코딩
img_raw = tf.image.decode_image(
    open(FLAGS.image, 'rb').read(), channels=3
)

# 배치 차원을 추가하여 모델에 맞게 확장
img = tf.expand_dims(img_raw, 0)
# 이미지를 모델 입력 크기로 변환 (예: 416x416)
img = transform_images(img, FLAGS.size)

t1 = time.time()  # 예측 시작 시간 기록
# 이미지에 대해 예측 수행
boxes, scores, classes, nums = yolo(img)
t2 = time.time()  # 예측 종료 시간 기록
logging.info('time: {}'.format(t2 - t1))  # 예측에 걸린 시간 로그에 기록

logging.info('detections:')  # 탐지된 객체를 로그에 기록
for i in range(nums[0]):
    logging.info('\t{}, {}, {}'.format(class_names[int(classes[0][i])],
                                       np.array(scores[0][i]),
                                       np.array(boxes[0][i])))

In [ ]:
# 이미지 색상 변환 (RGB에서 BGR로 변환하여 OpenCV에서 사용)
img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
# 예측된 바운딩 박스와 클래스 이름을 이미지에 그림
img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
# 결과 이미지를 파일로 저장
cv2.imwrite(FLAGS.output, img)
logging.info('output saved to: {}'.format(FLAGS.output))  # 결과 이미지 저장 로그

# Jupyter Notebook에서 이미지를 표시
from IPython.display import Image, display
display(Image(data=bytes(cv2.imencode('.jpg', img)[1]), width=800))